[RNN によるテキスト生成  \|  TensorFlow Core](https://www.tensorflow.org/tutorials/text/text_generation?hl=ja)

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

### シェイクスピアデータセットのダウンロード

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# ファイル中のユニークな文字の数
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


### テキストのベクトル化

In [5]:
# それぞれの文字からインデックスへの対応表を作成
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
print(char2idx)

{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64}


### 予測タスク

ある文字、あるいは文字列が与えられたとき、もっともありそうな次の文字はなにか？

### 訓練用サンプルとターゲットを作成

`seq_length` 個の文字があるとき、その次に現れる文字は何か？

例えば、テキストが「Hello」、seq_length が 4 だとすると、入力データが「Hell」で教師データが「ello」となる。

In [7]:
# ひとつの入力としたいシーケンスの文字数としての最大の長さ
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# 訓練用サンプルとターゲットを作る
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [8]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [9]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

最初のサンプルの入力とターゲット

In [10]:
for input_example, target_example in dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### 訓練用バッチの作成

In [11]:
# バッチサイズ
BATCH_SIZE = 64

# データセットをシャッフルするためのバッファサイズ
# （TF data は可能性として無限長のシーケンスでも使えるように設計されています。
# このため、シーケンス全体をメモリ内でシャッフルしようとはしません。
# その代わりに、要素をシャッフルするためのバッファを保持しています）
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

### モデルの構築

- Embedding: 入力層。それぞれの文字を表す数を embedding_dim 次元のベクトルに変換する、訓練可能な参照テーブル
- GRU : サイズが units=rnn_units のRNNの一種（ここに LSTM レイヤーを使うこともできる）（GRUはLSTMに比べて学習パラメータが少なく、計算時間が短い特徴がある）
- Dense: vocab_size 個の出力を持つ、出力層

出力の形は `(バッチサイズ, 文字列の長さ, 語彙数) = (64, 100, 65)` だが、モデルはどのような長さの入力もできるので、`(64, None, 65)` となる。

In [12]:
# 文字数で表されるボキャブラリーの長さ
vocab_size = len(vocab)

# 埋め込みベクトルの次元
embedding_dim = 256

# RNN ユニットの数
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


モデルから実際の予測を得るには出力の分布からサンプリングを行う必要がある。
分布からサンプリングのは、分布の argmax をとっただけでは、モデルは簡単にループしてしまうから。

In [13]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)

# 次の文字をランダムに選ぶ (サンプリング)
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
# Flatten
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices

array([45, 34, 24,  3, 23, 57, 30, 55,  4, 45, 41, 38, 51,  1, 34, 44, 23,
       35,  7, 40, 34, 61, 16, 26, 33, 53, 27,  0, 30, 57, 25, 48, 23, 49,
       24, 11,  9, 27, 13, 59, 29, 21, 52, 11, 63,  1, 10, 57, 50,  8, 22,
       64, 47, 29, 58, 22, 53, 34, 32, 23, 57, 61, 32, 52,  1, 18, 16, 18,
       30, 29,  1, 12, 17,  3, 22, 64,  9, 58, 41, 24, 44, 21,  0, 47, 60,
        5, 28, 49, 52, 36, 42, 60, 15, 48, 41, 22, 48, 48, 29, 63])

訓練前のモデルによる予測テキストをデコードする

In [14]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'icularly and to all the Volsces\nGreat hurt and mischief; thereto witness may\nMy surname, Coriolanus:'

Next Char Predictions: 
 "gVL$KsRq&gcZm VfKW-bVwDNUoO\nRsMjKkL;3OAuQIn;y :sl.JziQtJoVTKswTn FDFRQ ?E$Jz3tcLfI\niv'PknXdvCjcJjjQy"


### モデルの訓練

- 損失関数 : `from_logits=True` でロジスティック回帰 (0〜1にする?)

In [15]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1755595


In [16]:
model.compile(optimizer='adam', loss=loss)

### チェックポイントの構成

訓練中にチェックポイントを保存するようにする

In [17]:
# チェックポイントが保存されるディレクトリ
checkpoint_dir = './training_checkpoints'
# チェックポイントファイルの名称
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### モデルの訓練

In [18]:
EPOCHS = 10
history = model.fit(dataset, 
                    epochs=EPOCHS, 
                    callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/10
172/172 [==============================] - 11s 62ms/step - loss: 2.6526
Epoch 2/10
172/172 [==============================] - 9s 55ms/step - loss: 1.9388
Epoch 3/10
172/172 [==============================] - 9s 55ms/step - loss: 1.6744
Epoch 4/10
172/172 [==============================] - 9s 55ms/step - loss: 1.5302
Epoch 5/10
172/172 [==============================] - 9s 54ms/step - loss: 1.4447
Epoch 6/10
172/172 [==============================] - 9s 55ms/step - loss: 1.3861
Epoch 7/10
172/172 [==============================] - 9s 55ms/step - loss: 1.3412
Epoch 8/10
172/172 [==============================] - 9s 55ms/step - loss: 1.3006
Epoch 9/10
172/172 [==============================] - 10s 59ms/step - loss: 1.2669
Epoch 10/10
172/172 [==============================] - 9s 55ms/step - loss: 1.2324


## テキスト生成

### 最終チェックポイントの復元

予測ステップを単純にするため、バッチサイズ 1 を使用する。

RNN が状態をタイムステップからタイムステップへと渡す仕組みのため、モデルは一度構築されると固定されたバッチサイズしか受け付けないため、モデルを異なる batch_size で実行するためには、モデルを再構築し、チェックポイントから重みを復元する必要がある。

In [19]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [20]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None])) # 入力の形

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


### 予測ループ

1. 開始文字列を選択し、RNN の状態を初期化して、生成する文字数を設定する
2. 開始文字列と RNN の状態を使って、次の文字の予測分布を取得する
3. 予測分布から次の文字を求め、この文字をモデルの次の入力にする
4. モデルによって返された RNN の状態はモデルにフィードバックされるため、1つの文字だけでなく、より多くのコンテキストを持つことになる

In [21]:
def generate_text(model, start_string):
  # 評価ステップ（学習済みモデルを使ったテキスト生成）

  # 生成する文字数
  num_generate = 1000

  # 開始文字列を数値に変換（ベクトル化）
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # 結果を保存する空文字列
  text_generated = []

  # 低い temperature　は、より予測しやすいテキストをもたらし
  # 高い temperature は、より意外なテキストをもたらす
  # 実験により最適な設定を見つけること
  temperature = 1.0

  # ここではバッチサイズ　== 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # バッチの次元を削除
      predictions = tf.squeeze(predictions, 0)

      # カテゴリー分布をつかってモデルから返された文字を予測 
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # 過去の隠れ状態とともに予測された文字をモデルへのつぎの入力として渡す
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [22]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: no measure,
But his noble cousin?

Second Keeper:
But where he knew not that knows not this to leath which I do not much
doing in llin person, and her falsehear's makes this much:
I'll kiss my speech ank worship to God Eew purlips;
Without my uncles from his boundain than thousain?
Here refaise the time amongst the Volsces and my shoulder,
Or else a heavenly sprink in honset.
Good one fearful,
I'll bear your husband cries 'God save this? Was a much son,
Not let him ne'er down this newly heard;
Gromits, recapery are dreams;
We met, as he did such answer from a sight.

KATHARINA:
Will't Warwick?

ANGELO:
Itwas not but within my grave:
If I smoater, weapons of these namies,
From traitors oath lies: therefore shall you this is traitor.

VOLUMNIA:
Here, make her sport, if he be march'd together?
Alas! please you, sir: I sprinkshine, she shall be strange
Myself concealing his grief and let your grave.

HENRY BOLINGBROKE:
It was his heart; I had as law makes the ooth of his,
And dable 